# AMEDA - Angular Momentum Eddy Detection Algorithm
## Visualization and Analysis Notebook

This notebook demonstrates the Python implementation of AMEDA for detecting ocean eddies.

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.patches import Circle
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
warnings.filterwarnings('ignore')

# Import AMEDA modules
from ameda.params import AMEDAParams, compute_derived_params
from ameda.fields import process_fields, load_fields_aviso
from ameda.eddy_centers import detect_centers

print("AMEDA modules loaded successfully!")

## 1. Load and Explore Data

In [ ]:
# Dataset path
DATASET = "/Users/gianlucacalo/Desktop/projects/ocean_surface_levels/data/raw/cmems_obs-sl_eur_phy-ssh_my_allsat-l4-duacs-0.0625deg_P1D.nc"

# Load dataset
ds = xr.open_dataset(DATASET)

# Display dataset info
print("Dataset Information:")
print(f"Time range: {ds.time.values[0]} to {ds.time.values[-1]}")
print(f"Spatial extent: Lon [{ds.longitude.min().values:.1f}, {ds.longitude.max().values:.1f}]")
print(f"                Lat [{ds.latitude.min().values:.1f}, {ds.latitude.max().values:.1f}]")
print(f"Variables: {list(ds.data_vars.keys())}")
print(f"Grid resolution: {ds.longitude.values[1] - ds.longitude.values[0]:.4f}°")

## 2. Configure AMEDA Parameters

In [ ]:
# Configure parameters
params = AMEDAParams(
    x_name="longitude",
    y_name="latitude",
    u_name="ugos",
    v_name="vgos",
    s_name="sla",
    type_detection=3,  # Use both velocity and SSH
    K=0.5,  # LNAM threshold
    resol=1,  # No interpolation
    nRmin=0.5,  # Minimum eddy radius (in grid cells)
    nR_lim=100,  # Maximum eddy radius (in Rossby radii)
    lat_min=20  # Minimum latitude for detection
)

print("AMEDA Parameters:")
print(f"  Detection type: {'Velocity + SSH' if params.type_detection == 3 else 'Velocity only'}")
print(f"  LNAM threshold (K): {params.K}")
print(f"  Min latitude: {params.lat_min}°")
print(f"  Resolution factor: {params.resol}")

## 3. Select Time Step and Compute Derived Parameters

In [ ]:
# Select time step
step = 0  # First time step
data_step = ds.isel(time=step)

# Extract fields
lon = data_step.longitude.values
lat = data_step.latitude.values
ugos = data_step.ugos.values
vgos = data_step.vgos.values
sla = data_step.sla.values
mask = ~np.isnan(ugos)

# Compute derived parameters
derived_params = compute_derived_params(lon, lat, mask, ugos, vgos, params)

print(f"Processing time: {data_step.time.values}")
print(f"\nDerived parameters computed:")
print(f"  Rossby radius range: {np.nanmin(derived_params.Rd):.1f} - {np.nanmax(derived_params.Rd):.1f} km")
print(f"  Grid spacing: {np.nanmean(derived_params.Dx):.1f} km")
print(f"  LNAM box size: {np.nanmin(derived_params.b):.0f} - {np.nanmax(derived_params.b):.0f} pixels")

## 4. Visualize Input Fields

In [ ]:
# Create figure with input fields
fig = plt.figure(figsize=(18, 10))

# Create meshgrid
lon_grid, lat_grid = np.meshgrid(lon, lat)

# 1. Sea Level Anomaly
ax1 = fig.add_subplot(2, 3, 1, projection=ccrs.PlateCarree())
im1 = ax1.pcolormesh(lon_grid, lat_grid, sla, 
                     cmap='RdBu_r', vmin=-0.1, vmax=0.1,
                     transform=ccrs.PlateCarree())
ax1.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax1.add_feature(cfeature.LAND, facecolor='lightgray')
ax1.set_title('Sea Level Anomaly (m)')
plt.colorbar(im1, ax=ax1, orientation='horizontal', pad=0.05, shrink=0.8)

# 2. Zonal velocity (u)
ax2 = fig.add_subplot(2, 3, 2, projection=ccrs.PlateCarree())
im2 = ax2.pcolormesh(lon_grid, lat_grid, ugos,
                     cmap='RdBu_r', vmin=-0.3, vmax=0.3,
                     transform=ccrs.PlateCarree())
ax2.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax2.add_feature(cfeature.LAND, facecolor='lightgray')
ax2.set_title('Zonal Velocity (m/s)')
plt.colorbar(im2, ax=ax2, orientation='horizontal', pad=0.05, shrink=0.8)

# 3. Meridional velocity (v)
ax3 = fig.add_subplot(2, 3, 3, projection=ccrs.PlateCarree())
im3 = ax3.pcolormesh(lon_grid, lat_grid, vgos,
                     cmap='RdBu_r', vmin=-0.3, vmax=0.3,
                     transform=ccrs.PlateCarree())
ax3.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax3.add_feature(cfeature.LAND, facecolor='lightgray')
ax3.set_title('Meridional Velocity (m/s)')
plt.colorbar(im3, ax=ax3, orientation='horizontal', pad=0.05, shrink=0.8)

# 4. Velocity magnitude
speed = np.sqrt(ugos**2 + vgos**2)
ax4 = fig.add_subplot(2, 3, 4, projection=ccrs.PlateCarree())
im4 = ax4.pcolormesh(lon_grid, lat_grid, speed,
                     cmap='viridis', vmin=0, vmax=0.4,
                     transform=ccrs.PlateCarree())
ax4.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax4.add_feature(cfeature.LAND, facecolor='lightgray')
ax4.set_title('Speed (m/s)')
plt.colorbar(im4, ax=ax4, orientation='horizontal', pad=0.05, shrink=0.8)

# 5. Rossby radius
ax5 = fig.add_subplot(2, 3, 5, projection=ccrs.PlateCarree())
im5 = ax5.pcolormesh(lon_grid, lat_grid, derived_params.Rd,
                     cmap='plasma', vmin=20, vmax=60,
                     transform=ccrs.PlateCarree())
ax5.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax5.add_feature(cfeature.LAND, facecolor='lightgray')
ax5.set_title('First Baroclinic Rossby Radius (km)')
plt.colorbar(im5, ax=ax5, orientation='horizontal', pad=0.05, shrink=0.8)

# 6. Velocity vectors (subsampled)
ax6 = fig.add_subplot(2, 3, 6, projection=ccrs.PlateCarree())
skip = 5  # Subsample for clarity
ax6.quiver(lon_grid[::skip, ::skip], lat_grid[::skip, ::skip],
           ugos[::skip, ::skip], vgos[::skip, ::skip],
           speed[::skip, ::skip], cmap='viridis', scale=2,
           transform=ccrs.PlateCarree())
ax6.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax6.add_feature(cfeature.LAND, facecolor='lightgray')
ax6.set_title('Velocity Vectors')

plt.suptitle(f'Input Fields - {data_step.time.values}', fontsize=14, y=0.98)
plt.tight_layout()
plt.show()

## 5. Compute Detection Fields

In [ ]:
# Compute detection fields
print("Computing detection fields...")
fields = process_fields(DATASET, step, params, derived_params)

print("\nField statistics:")
print(f"  Kinetic Energy: {np.nanmin(fields.ke):.6f} - {np.nanmax(fields.ke):.6f} m²/s²")
print(f"  Vorticity: {np.nanmin(fields.vort):.2e} - {np.nanmax(fields.vort):.2e} s⁻¹")
print(f"  Okubo-Weiss: {np.nanmin(fields.OW):.2e} - {np.nanmax(fields.OW):.2e} s⁻²")
print(f"  Local OW: {np.nanmin(fields.LOW):.2e} - {np.nanmax(fields.LOW):.2e} s⁻²")
print(f"  LNAM: {np.nanmin(fields.LNAM):.3f} - {np.nanmax(fields.LNAM):.3f}")

## 6. Visualize Detection Fields

In [ ]:
# Create figure with detection fields
fig = plt.figure(figsize=(18, 10))

# Load fields for grid
x, y, mask_field, _, _, _ = load_fields_aviso(ds, step, params, params.resol, params.deg)

# 1. Kinetic Energy
ax1 = fig.add_subplot(2, 3, 1, projection=ccrs.PlateCarree())
im1 = ax1.pcolormesh(x, y, fields.ke,
                     cmap='YlOrRd', vmin=0, vmax=0.02,
                     transform=ccrs.PlateCarree())
ax1.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax1.add_feature(cfeature.LAND, facecolor='lightgray')
ax1.set_title('Kinetic Energy (m²/s²)')
plt.colorbar(im1, ax=ax1, orientation='horizontal', pad=0.05, shrink=0.8)

# 2. Vorticity
ax2 = fig.add_subplot(2, 3, 2, projection=ccrs.PlateCarree())
vort_lim = 1e-5
im2 = ax2.pcolormesh(x, y, fields.vort,
                     cmap='RdBu_r', vmin=-vort_lim, vmax=vort_lim,
                     transform=ccrs.PlateCarree())
ax2.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax2.add_feature(cfeature.LAND, facecolor='lightgray')
ax2.set_title('Vorticity (s⁻¹)')
plt.colorbar(im2, ax=ax2, orientation='horizontal', pad=0.05, shrink=0.8, format='%.1e')

# 3. Okubo-Weiss
ax3 = fig.add_subplot(2, 3, 3, projection=ccrs.PlateCarree())
ow_lim = 1e-10
im3 = ax3.pcolormesh(x, y, fields.OW,
                     cmap='RdBu_r', vmin=-ow_lim, vmax=ow_lim,
                     transform=ccrs.PlateCarree())
ax3.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax3.add_feature(cfeature.LAND, facecolor='lightgray')
ax3.set_title('Okubo-Weiss (s⁻²)')
plt.colorbar(im3, ax=ax3, orientation='horizontal', pad=0.05, shrink=0.8, format='%.1e')

# 4. Local Okubo-Weiss
ax4 = fig.add_subplot(2, 3, 4, projection=ccrs.PlateCarree())
im4 = ax4.pcolormesh(x, y, fields.LOW,
                     cmap='RdBu_r', vmin=-ow_lim, vmax=ow_lim,
                     transform=ccrs.PlateCarree())
ax4.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax4.add_feature(cfeature.LAND, facecolor='lightgray')
ax4.set_title('Local Okubo-Weiss (s⁻²)')
plt.colorbar(im4, ax=ax4, orientation='horizontal', pad=0.05, shrink=0.8, format='%.1e')

# 5. LNAM
ax5 = fig.add_subplot(2, 3, 5, projection=ccrs.PlateCarree())
im5 = ax5.pcolormesh(x, y, fields.LNAM,
                     cmap='RdBu_r', vmin=-1, vmax=1,
                     transform=ccrs.PlateCarree())
ax5.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax5.add_feature(cfeature.LAND, facecolor='lightgray')
ax5.set_title('LNAM (Local Normalized Angular Momentum)')
plt.colorbar(im5, ax=ax5, orientation='horizontal', pad=0.05, shrink=0.8)

# 6. Potential eddy regions
ax6 = fig.add_subplot(2, 3, 6, projection=ccrs.PlateCarree())
potential = np.abs(fields.LNAM) * (fields.LOW < 0)
im6 = ax6.pcolormesh(x, y, potential,
                     cmap='hot_r', vmin=0, vmax=1,
                     transform=ccrs.PlateCarree())
ax6.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax6.add_feature(cfeature.LAND, facecolor='lightgray')
ax6.contour(x, y, potential, levels=[params.K], colors='cyan', linewidths=1)
ax6.set_title(f'Potential Eddy Regions (|LNAM| with LOW<0, contour at K={params.K})')
plt.colorbar(im6, ax=ax6, orientation='horizontal', pad=0.05, shrink=0.8)

plt.suptitle('Detection Fields', fontsize=14, y=0.98)
plt.tight_layout()
plt.show()

## 7. Detect Eddy Centers

In [ ]:
# Simple center detection (without full streamline validation for speed)
print("Detecting eddy centers...")

# Find LNAM maxima where LOW < 0
LOW = fields.LOW
LNAM = fields.LNAM
abs_lnam = np.abs(LNAM)

# Apply thresholds
candidates = (abs_lnam > params.K) & (LOW < 0) & (mask_field > 0)

# Apply latitude constraint if using geographic coordinates
if params.grid_ll and params.lat_min > 0:
    candidates = candidates & (np.abs(y) > params.lat_min)

# Find local maxima
from scipy.ndimage import maximum_filter
local_max = (abs_lnam == maximum_filter(abs_lnam, size=3)) & candidates

# Get center coordinates
j_idx, i_idx = np.where(local_max)
n_centers = len(j_idx)

print(f"\nFound {n_centers} potential eddy centers")

# Classify eddies
cyclonic = []
anticyclonic = []

for k in range(n_centers):
    center_type = np.sign(LNAM[j_idx[k], i_idx[k]])
    center_info = {
        'lon': x[j_idx[k], i_idx[k]],
        'lat': y[j_idx[k], i_idx[k]],
        'lnam': LNAM[j_idx[k], i_idx[k]],
        'i': i_idx[k],
        'j': j_idx[k]
    }
    
    if center_type > 0:
        cyclonic.append(center_info)
    else:
        anticyclonic.append(center_info)

print(f"  Cyclonic eddies: {len(cyclonic)}")
print(f"  Anticyclonic eddies: {len(anticyclonic)}")

# Show first few centers
print("\nFirst 5 cyclonic centers:")
for i, c in enumerate(cyclonic[:5]):
    print(f"  {i+1}. Lon: {c['lon']:.3f}°, Lat: {c['lat']:.3f}°, LNAM: {c['lnam']:.3f}")

print("\nFirst 5 anticyclonic centers:")
for i, c in enumerate(anticyclonic[:5]):
    print(f"  {i+1}. Lon: {c['lon']:.3f}°, Lat: {c['lat']:.3f}°, LNAM: {c['lnam']:.3f}")

## 8. Visualize Detected Eddies

In [ ]:
# Create comprehensive eddy visualization
fig = plt.figure(figsize=(20, 8))

# 1. Eddies on SSH field
ax1 = fig.add_subplot(1, 3, 1, projection=ccrs.PlateCarree())
im1 = ax1.pcolormesh(lon_grid, lat_grid, sla,
                     cmap='RdBu_r', vmin=-0.1, vmax=0.1,
                     transform=ccrs.PlateCarree(), alpha=0.8)

# Plot eddy centers
for c in cyclonic:
    ax1.plot(c['lon'], c['lat'], 'b^', markersize=6, transform=ccrs.PlateCarree())
for c in anticyclonic:
    ax1.plot(c['lon'], c['lat'], 'rv', markersize=6, transform=ccrs.PlateCarree())

ax1.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax1.add_feature(cfeature.LAND, facecolor='lightgray')
ax1.set_title('Detected Eddies on SSH\n(Blue ▲: Cyclonic, Red ▼: Anticyclonic)')
plt.colorbar(im1, ax=ax1, orientation='horizontal', pad=0.05, shrink=0.8, label='SSH (m)')

# 2. Eddies on LNAM field
ax2 = fig.add_subplot(1, 3, 2, projection=ccrs.PlateCarree())
im2 = ax2.pcolormesh(x, y, fields.LNAM,
                     cmap='RdBu_r', vmin=-1.5, vmax=1.5,
                     transform=ccrs.PlateCarree(), alpha=0.8)

# Plot eddy centers
for c in cyclonic:
    ax2.plot(c['lon'], c['lat'], 'b^', markersize=6, transform=ccrs.PlateCarree())
for c in anticyclonic:
    ax2.plot(c['lon'], c['lat'], 'rv', markersize=6, transform=ccrs.PlateCarree())

# Add LNAM threshold contours
ax2.contour(x, y, np.abs(fields.LNAM), levels=[params.K], colors='green', linewidths=1, alpha=0.5)

ax2.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax2.add_feature(cfeature.LAND, facecolor='lightgray')
ax2.set_title(f'LNAM Field with Centers\n(Green contour: |LNAM|={params.K})')
plt.colorbar(im2, ax=ax2, orientation='horizontal', pad=0.05, shrink=0.8, label='LNAM')

# 3. Eddies on velocity field
ax3 = fig.add_subplot(1, 3, 3, projection=ccrs.PlateCarree())

# Velocity magnitude background
speed = np.sqrt(ugos**2 + vgos**2)
im3 = ax3.pcolormesh(lon_grid, lat_grid, speed,
                     cmap='YlOrRd', vmin=0, vmax=0.3,
                     transform=ccrs.PlateCarree(), alpha=0.6)

# Velocity vectors
skip = 4
ax3.quiver(lon_grid[::skip, ::skip], lat_grid[::skip, ::skip],
           ugos[::skip, ::skip], vgos[::skip, ::skip],
           scale=3, alpha=0.5, transform=ccrs.PlateCarree())

# Plot eddy centers with size proportional to |LNAM|
for c in cyclonic:
    size = np.abs(c['lnam']) * 50
    ax3.scatter(c['lon'], c['lat'], s=size, c='blue', marker='^', 
               edgecolors='darkblue', alpha=0.7, transform=ccrs.PlateCarree())
for c in anticyclonic:
    size = np.abs(c['lnam']) * 50
    ax3.scatter(c['lon'], c['lat'], s=size, c='red', marker='v',
               edgecolors='darkred', alpha=0.7, transform=ccrs.PlateCarree())

ax3.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax3.add_feature(cfeature.LAND, facecolor='lightgray')
ax3.set_title('Velocity Field with Eddies\n(Size ∝ |LNAM|)')
plt.colorbar(im3, ax=ax3, orientation='horizontal', pad=0.05, shrink=0.8, label='Speed (m/s)')

plt.suptitle(f'Eddy Detection Results - {data_step.time.values}', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

# Summary statistics
print("\nEddy Statistics:")
print(f"Total eddies detected: {n_centers}")
print(f"  Cyclonic: {len(cyclonic)} ({len(cyclonic)/n_centers*100:.1f}%)")
print(f"  Anticyclonic: {len(anticyclonic)} ({len(anticyclonic)/n_centers*100:.1f}%)")

if cyclonic:
    cyc_lnam = [abs(c['lnam']) for c in cyclonic]
    print(f"\nCyclonic LNAM: {np.mean(cyc_lnam):.3f} ± {np.std(cyc_lnam):.3f}")
    print(f"  Range: [{np.min(cyc_lnam):.3f}, {np.max(cyc_lnam):.3f}]")

if anticyclonic:
    anti_lnam = [abs(c['lnam']) for c in anticyclonic]
    print(f"\nAnticyclonic LNAM: {np.mean(anti_lnam):.3f} ± {np.std(anti_lnam):.3f}")
    print(f"  Range: [{np.min(anti_lnam):.3f}, {np.max(anti_lnam):.3f}]")

## 9. Regional Analysis

In [ ]:
# Define region of interest (example: Western Mediterranean)
lon_min, lon_max = -5, 10
lat_min, lat_max = 35, 44

# Filter eddies in region
regional_cyclonic = [c for c in cyclonic 
                     if lon_min <= c['lon'] <= lon_max and lat_min <= c['lat'] <= lat_max]
regional_anticyclonic = [c for c in anticyclonic 
                         if lon_min <= c['lon'] <= lon_max and lat_min <= c['lat'] <= lat_max]

print(f"Regional Analysis: [{lon_min}°, {lon_max}°] × [{lat_min}°, {lat_max}°]")
print(f"  Cyclonic: {len(regional_cyclonic)}")
print(f"  Anticyclonic: {len(regional_anticyclonic)}")

# Create zoomed map
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Set extent
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# Plot SSH
im = ax.pcolormesh(lon_grid, lat_grid, sla,
                   cmap='RdBu_r', vmin=-0.1, vmax=0.1,
                   transform=ccrs.PlateCarree(), alpha=0.8)

# Plot velocity vectors
skip = 3
q = ax.quiver(lon_grid[::skip, ::skip], lat_grid[::skip, ::skip],
              ugos[::skip, ::skip], vgos[::skip, ::skip],
              speed[::skip, ::skip], cmap='gray', scale=2, alpha=0.5,
              transform=ccrs.PlateCarree())

# Plot regional eddies
for c in regional_cyclonic:
    ax.plot(c['lon'], c['lat'], 'b^', markersize=10, 
           markeredgecolor='darkblue', markeredgewidth=1.5,
           transform=ccrs.PlateCarree())
    
for c in regional_anticyclonic:
    ax.plot(c['lon'], c['lat'], 'rv', markersize=10,
           markeredgecolor='darkred', markeredgewidth=1.5,
           transform=ccrs.PlateCarree())

# Add features
ax.add_feature(cfeature.COASTLINE, linewidth=1)
ax.add_feature(cfeature.LAND, facecolor='lightgray')
ax.add_feature(cfeature.OCEAN, facecolor='white', alpha=0.3)
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.3)

plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02, shrink=0.8, label='SSH (m)')
ax.set_title(f'Regional Eddy Detection\nCyclonic: {len(regional_cyclonic)}, Anticyclonic: {len(regional_anticyclonic)}')
plt.tight_layout()
plt.show()

## 10. Time Series Analysis (Multiple Steps)

In [ ]:
# Process multiple time steps
n_steps = min(10, len(ds.time))  # Process first 10 days
eddy_counts = {'cyclonic': [], 'anticyclonic': [], 'time': []}

print(f"Processing {n_steps} time steps...")

for step in range(n_steps):
    # Get data
    data_step = ds.isel(time=step)
    ugos = data_step.ugos.values
    vgos = data_step.vgos.values
    mask = ~np.isnan(ugos)
    
    # Process fields
    try:
        fields = process_fields(DATASET, step, params, derived_params)
        
        # Simple center detection
        LOW = fields.LOW
        LNAM = fields.LNAM
        abs_lnam = np.abs(LNAM)
        
        # Find centers
        candidates = (abs_lnam > params.K) & (LOW < 0) & (mask > 0)
        if params.grid_ll:
            candidates = candidates & (np.abs(y) > params.lat_min)
        
        local_max = (abs_lnam == maximum_filter(abs_lnam, size=3)) & candidates
        j_idx, i_idx = np.where(local_max)
        
        # Count by type
        n_cyc = np.sum(LNAM[j_idx, i_idx] > 0)
        n_anti = np.sum(LNAM[j_idx, i_idx] < 0)
        
        eddy_counts['cyclonic'].append(n_cyc)
        eddy_counts['anticyclonic'].append(n_anti)
        eddy_counts['time'].append(data_step.time.values)
        
        print(f"  Step {step}: {n_cyc} cyclonic, {n_anti} anticyclonic")
        
    except Exception as e:
        print(f"  Step {step}: Error - {e}")
        eddy_counts['cyclonic'].append(0)
        eddy_counts['anticyclonic'].append(0)
        eddy_counts['time'].append(data_step.time.values)

# Plot time series
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Eddy counts
ax1.plot(eddy_counts['time'], eddy_counts['cyclonic'], 'b-o', label='Cyclonic')
ax1.plot(eddy_counts['time'], eddy_counts['anticyclonic'], 'r-s', label='Anticyclonic')
ax1.plot(eddy_counts['time'], 
         np.array(eddy_counts['cyclonic']) + np.array(eddy_counts['anticyclonic']),
         'k--', alpha=0.5, label='Total')
ax1.set_ylabel('Number of Eddies')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_title('Eddy Detection Time Series')

# Ratio
total = np.array(eddy_counts['cyclonic']) + np.array(eddy_counts['anticyclonic'])
ratio = np.array(eddy_counts['cyclonic']) / np.maximum(total, 1) * 100
ax2.bar(eddy_counts['time'], ratio, color='blue', alpha=0.6, label='Cyclonic %')
ax2.axhline(50, color='gray', linestyle='--', alpha=0.5)
ax2.set_ylabel('Cyclonic Percentage (%)')
ax2.set_xlabel('Time')
ax2.set_ylim([0, 100])
ax2.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Statistics
print(f"\nTime Series Statistics ({n_steps} days):")
print(f"  Mean cyclonic: {np.mean(eddy_counts['cyclonic']):.1f} ± {np.std(eddy_counts['cyclonic']):.1f}")
print(f"  Mean anticyclonic: {np.mean(eddy_counts['anticyclonic']):.1f} ± {np.std(eddy_counts['anticyclonic']):.1f}")
print(f"  Mean total: {np.mean(total):.1f} ± {np.std(total):.1f}")

## 11. Export Results

In [ ]:
# Create a summary dataset with detected eddies
import pandas as pd

# Convert to DataFrame for easy export
if cyclonic or anticyclonic:
    eddy_data = []
    
    for c in cyclonic:
        eddy_data.append({
            'type': 'cyclonic',
            'lon': c['lon'],
            'lat': c['lat'],
            'lnam': c['lnam'],
            'time': ds.time.values[0]
        })
    
    for c in anticyclonic:
        eddy_data.append({
            'type': 'anticyclonic',
            'lon': c['lon'],
            'lat': c['lat'],
            'lnam': c['lnam'],
            'time': ds.time.values[0]
        })
    
    df_eddies = pd.DataFrame(eddy_data)
    
    print("Eddy Detection Summary:")
    print(df_eddies.head(10))
    print(f"\nTotal eddies: {len(df_eddies)}")
    print(df_eddies.groupby('type').agg({
        'lnam': ['count', 'mean', 'std', 'min', 'max']
    }))
    
    # Optional: Save to CSV
    # df_eddies.to_csv('detected_eddies.csv', index=False)
    # print("\nResults saved to detected_eddies.csv")

# Close dataset
ds.close()
print("\nAnalysis complete!")